In [2]:
from ultralytics import YOLO
import cv2
import numpy as np

# EOTS CONFIGURATION
# Horizontal & Vertical FOV (degrees)
FOV_H = 4.0
FOV_V = 3.0

# Gimbal LOS angles (degrees)
GIMBAL_AZ = 125.35
GIMBAL_EL = 12.80

# VIDEO INPUT / OUTPUT

input_video = r"C:\aerial_project\test_video\test4.mp4"
output_video = r"C:\aerial_project\output\output2.mp4"

cap = cv2.VideoCapture(input_video)

# Get video properties
IMG_W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
IMG_H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (IMG_W, IMG_H))

cap.release()   # We only used it to read video properties

# HELPER FUNCTIONS

def compute_centroid_xy(x1, y1, x2, y2):
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    return cx, cy


def pixel_to_angle(cx, cy):
    # Image center
    cx0 = IMG_W / 2.0
    cy0 = IMG_H / 2.0

    # Pixel offset from center
    dx = cx - cx0
    dy = cy0 - cy  # Inverted Y-axis

    # Degree per pixel
    deg_per_pix_h = FOV_H / IMG_W
    deg_per_pix_v = FOV_V / IMG_H

    # Angle offsets
    d_az = dx * deg_per_pix_h
    d_el = dy * deg_per_pix_v

    return d_az, d_el


def compute_absolute_az_el(cx, cy):
    d_az, d_el = pixel_to_angle(cx, cy)
    az = GIMBAL_AZ + d_az
    el = GIMBAL_EL + d_el
    return az, el

# LOAD YOLO MODEL

model = YOLO(r"C:\aerial_project\yolov8n.pt")

# YOLO TRACKING LOOP

results = model.track(
    source=input_video,
    tracker="bytetrack.yaml",
    persist=True,
    stream=True
)

for r in results:

    frame = r.orig_img

    if r.boxes.id is not None:
        boxes = r.boxes.xyxy.cpu().numpy()
        track_ids = r.boxes.id.cpu().numpy()

        for box, tid in zip(boxes, track_ids):

            x1, y1, x2, y2 = map(int, box)

            # Compute centroid
            cx, cy = compute_centroid_xy(x1, y1, x2, y2)

            # Compute Azimuth & Elevation
            az, el = compute_absolute_az_el(cx, cy)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Display label
            label = f"ID:{int(tid)}  Az:{az:.2f}  El:{el:.2f}"
            cv2.putText(
                frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2
            )

    # Write output
    out.write(frame)

    # Show live window
    cv2.imshow("EOTS Multi-Object Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


out.release()
cv2.destroyAllWindows()

print("Processing complete. Output saved as:", output_video)


video 1/1 (1/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 339.3ms
video 1/1 (2/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 315.5ms
video 1/1 (3/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 304.2ms
video 1/1 (4/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 308.1ms
video 1/1 (5/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 304.4ms
video 1/1 (6/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 299.0ms
video 1/1 (7/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 244.6ms
video 1/1 (8/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 182.1ms
video 1/1 (9/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 181.9ms
video 1/1 (10/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 152.5ms
video 1/1 (11/329) C:\aerial_project\test_video\test4.mp4: 512x640 (no detections), 193.

Processing complete. Output saved as: C:\aerial_project\output\output2.mp4
